In [1]:
import pandas as pd
import geemap
import ee

In [2]:
Map = geemap.Map(center=(29, 85))
Map

Map(center=[29, 85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [3]:
import os
os.chdir(r'H:\Monarch\Work\qizhong')

In [4]:
shp = geemap.shp_to_ee("roi.shp")
roi = shp.geometry()
Map.addLayer(roi, {'color':'red'}, 'roi')

In [5]:
Map.center_object(shp, 11)

In [12]:
def bitwise_extract(value: ee.Image, from_bit: int, to_bit: int = None) -> ee.Image:
    """

    Args:
        value: ee.Image, 输入Landsat去云波段
        from_bit: int, 云的位掩码
        to_bit: int, modis去云时需要的参数, 当掩码有多位时使用

    Returns: ee.Image

    """
    if to_bit is None:
        to_bit = from_bit
    mask_size = ee.Number(1).add(to_bit).subtract(from_bit)
    mask = ee.Number(1).leftShift(mask_size).subtract(1)
    return value.rightShift(from_bit).bitwiseAnd(mask)

def cloud_free_landsat_sr(img):
    """
    Landsat 去云处理
    Args:
        img: ee.Image, 原始影像

    Returns: ee.Image, 去云后的影像

    """
    qa = img.select('pixel_qa')                         # 特定的landsat 影像才有这一波段
#     qa = img.select('QA_PIXEL')
    cloud_state = bitwise_extract(qa, 5)                # 云掩码
    cloud_shadow_state = bitwise_extract(qa, 3)         # 云影掩码
    mask = cloud_state.eq(0).And(cloud_shadow_state.eq(0))
    return img.updateMask(mask)

In [20]:
images = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterDate('2019', '2020')\
        .filterBounds(roi).sort("CLOUD_COVER", False)\
        .map(cloud_free_landsat_sr)

In [21]:
image = images.mosaic().clip(roi)

In [16]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000,
  'gamma': 1.4}

In [17]:
Map.addLayer(image, visParams, 'img')

In [24]:
ndvi = image.normalizedDifference(['B5', 'B4'])

In [23]:
geemap.ee_export_image(image.select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']), r'Landsat.tif', scale=30)

Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\Work\qizhong\Landsat.tif


In [25]:
geemap.ee_export_image(ndvi, r'ndvi.tif', scale=30)

Generating URL ...
Please wait ...
Data downloaded to H:\Monarch\Work\qizhong\ndvi.tif


In [ ]:
points = geemap.pandas_to_ee(df, latitude='Y', longitude='X')
Map.addLayer(points, {}, 'ps')

In [4]:
roi = Map.draw_last_feature.geometry()

In [7]:
images = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
        .filterDate('2020-07', '2020-08')\
        .filterBounds(roi).sort("CLOUD_COVER", True)

In [13]:
images = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")\
        .filterDate('2020-07', '2020-08')\
        .filterBounds(roi).sort("CLOUD_COVER", True)

In [14]:
a = images.aggregate_array("CLOUD_COVER").getInfo()
a

[79.86]

In [ ]:
def rmCloudByQA(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
               .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

In [ ]:
def bitwiseExtract(value, fromBit, toBit=None):
    if toBit == None:
        toBit = fromBit
    maskSize = ee.Number(1).add(toBit).subtract(fromBit)
    mask = ee.Number(1).leftShift(maskSize).subtract(1)
    return value.rightShift(fromBit).bitwiseAnd(mask)

def cloudfree_landsat(img):
    qa = img.select('pixel_qa')
    cloudState = bitwiseExtract(qa, 5)
    cloudShadowState = bitwiseExtract(qa, 3)
    mask = cloudState.eq(0).And(cloudShadowState.eq(0))
    return img.updateMask(mask)

In [ ]:
cf_image = images.map(cloudfree_landsat)

In [ ]:
image = cf_image.mosaic().clip(roi)

In [ ]:
# img = images.median().select(['B4', 'B3', 'B2']).clip(roi)
Map.addLayer(image, {'bands':['B4', 'B3', 'B2'], 'max': 3000, 'min':0}, 'img_1')

In [ ]:
geemap.ee_export_image_to_drive(img, description='RGB', 
                                scale=30, folder='Landsat8_xizhang')
geemap.ee_export_image_to_drive(img, description='NDVI', 
                                scale=30, folder='Landsat8_xizhang')

In [ ]:
NDVI = image.normalizedDifference(['B5', 'B4'])
img = image.select(['B4', 'B3', 'B2'])

In [ ]:
geemap.ee_export_image_to_drive(img, description='RGB', 
                                scale=30, folder='Landsat8_xizhang')

In [ ]:
a = """
function rmCloudByQA(image) { 
  var qa = image.select('QA60'); 
  var cloudBitMask = 1 << 10; 
  var cirrusBitMask = 1 << 11; 
  var mask = qa.bitwiseAnd(cloudBitMask).eq(0) 
               .and(qa.bitwiseAnd(cirrusBitMask).eq(0)); 
  return image.updateMask(mask); 
} 
"""

In [ ]:
geemap.js_snippet_to_py(a)